# Student's t-test for significantly different variances in Julia:

In [1]:
using BenchmarkTools


# Import Julia functions from lib
include("../lib_stat/lib_stat.jl")
using .lib_stat: avevar, betai

# Compile fortran source code (using a Python script)
run(`python ../lib_stat/compile_fortran.py`)


data1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
data2 = [1.1, 2, 3, 4, 5, 6, 7, 8, 9];

Function definition:

In [2]:
function tutest(data1, data2)
    # Given the arrays "data1" and "data2", returns the
    # Student's "t" and its significance as "prob".
    # Data are assumed to be drawn from populations with
    # the same true variance.

    # Small values of "prob" indicates that the arrays 
    # have different means.
    n1 = length(data1)
    n2 = length(data2)
    
    ave1, var1 = avevar(data1)
    ave2, var2 = avevar(data2)

    # print(ave1, ave2)
    # print(var1, var2)

    # Student's t
    t = (ave1 - ave2) / sqrt(var1/n1 + var2/n2)

    df = (var1/n1 + var2/n2)^2 / ((var1/n1)^2 / (n1 - 1) + (var2/n2)^2 / (n2 - 1))
    # significance
    prob = betai(0.5 * df, 0.5, df/(df + t^2))
    
    return t, prob
end;

### Calling the ttest function in Julia:

In [3]:
t, prob = tutest(data1, data2)

print(t, " ", prob)

-0.008635140708713388 0.9932169972438553

## Timing against Fortran:

Test function:

In [4]:
function test()
    for i = 1:1e3
        tutest(data1, data2)
        end
    end;

#### Julia:

In [5]:
@btime test()

  399.422 μs (4000 allocations: 406.25 KiB)


#### Fortran:

In [6]:
@btime run(`../lib_stat/fmain_tutest`);

  1.651 ms (46 allocations: 1.84 KiB)
